In [1]:
# Dependencies and set up
import pandas as pd
import numpy as np
import json

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from flask import Flask, jsonify
from sqlalchemy import inspect

from sqlite3 import connect

# City of Chicago -- Crimes - from 2001 to 2021

In [3]:
# Raw crime table
table = pd.read_csv('Resources/Crimes_-_2001_to_Present.csv') 
table.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,10224738,HY411648,09/05/2015 01:30:00 PM,043XX S WOOD ST,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,...,12.0,61.0,08B,1165074.0,1875917.0,2015,02/10/2018 03:50:01 PM,41.815117,-87.670000,"(41.815117282, -87.669999562)"
1,10224739,HY411615,09/04/2015 11:30:00 AM,008XX N CENTRAL AVE,0870,THEFT,POCKET-PICKING,CTA BUS,False,False,...,29.0,25.0,06,1138875.0,1904869.0,2015,02/10/2018 03:50:01 PM,41.895080,-87.765400,"(41.895080471, -87.765400451)"
2,11646166,JC213529,09/01/2018 12:01:00 AM,082XX S INGLESIDE AVE,0810,THEFT,OVER $500,RESIDENCE,False,True,...,8.0,44.0,06,NaN,NaN,2018,04/06/2019 04:04:43 PM,NaN,NaN,NaN
3,10224740,HY411595,09/05/2015 12:45:00 PM,035XX W BARRY AVE,2023,NARCOTICS,POSS: HEROIN(BRN/TAN),SIDEWALK,True,False,...,35.0,21.0,18,1152037.0,1920384.0,2015,02/10/2018 03:50:01 PM,41.937406,-87.716650,"(41.937405765, -87.716649687)"
4,10224741,HY411610,09/05/2015 01:00:00 PM,0000X N LARAMIE AVE,0560,ASSAULT,SIMPLE,APARTMENT,False,True,...,28.0,25.0,08A,1141706.0,1900086.0,2015,02/10/2018 03:50:01 PM,41.881903,-87.755121,"(41.881903443, -87.755121152)"


In [5]:
# Defining table from 2011 to 2020
df = table[(table.Year>2010) & (table.Year<2021) & (table['Latitude'].notna())]
df.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,10224738,HY411648,09/05/2015 01:30:00 PM,043XX S WOOD ST,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,...,12.0,61.0,08B,1165074.0,1875917.0,2015,02/10/2018 03:50:01 PM,41.815117,-87.670000,"(41.815117282, -87.669999562)"
1,10224739,HY411615,09/04/2015 11:30:00 AM,008XX N CENTRAL AVE,0870,THEFT,POCKET-PICKING,CTA BUS,False,False,...,29.0,25.0,06,1138875.0,1904869.0,2015,02/10/2018 03:50:01 PM,41.895080,-87.765400,"(41.895080471, -87.765400451)"
3,10224740,HY411595,09/05/2015 12:45:00 PM,035XX W BARRY AVE,2023,NARCOTICS,POSS: HEROIN(BRN/TAN),SIDEWALK,True,False,...,35.0,21.0,18,1152037.0,1920384.0,2015,02/10/2018 03:50:01 PM,41.937406,-87.716650,"(41.937405765, -87.716649687)"
4,10224741,HY411610,09/05/2015 01:00:00 PM,0000X N LARAMIE AVE,0560,ASSAULT,SIMPLE,APARTMENT,False,True,...,28.0,25.0,08A,1141706.0,1900086.0,2015,02/10/2018 03:50:01 PM,41.881903,-87.755121,"(41.881903443, -87.755121152)"
5,10224742,HY411435,09/05/2015 10:55:00 AM,082XX S LOOMIS BLVD,0610,BURGLARY,FORCIBLE ENTRY,RESIDENCE,False,False,...,21.0,71.0,05,1168430.0,1850165.0,2015,02/10/2018 03:50:01 PM,41.744379,-87.658431,"(41.744378879, -87.658430635)"


In [6]:
# Reset index
df.reset_index(drop=True, inplace=True)
df.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,10224738,HY411648,09/05/2015 01:30:00 PM,043XX S WOOD ST,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,...,12.0,61.0,08B,1165074.0,1875917.0,2015,02/10/2018 03:50:01 PM,41.815117,-87.670000,"(41.815117282, -87.669999562)"
1,10224739,HY411615,09/04/2015 11:30:00 AM,008XX N CENTRAL AVE,0870,THEFT,POCKET-PICKING,CTA BUS,False,False,...,29.0,25.0,06,1138875.0,1904869.0,2015,02/10/2018 03:50:01 PM,41.895080,-87.765400,"(41.895080471, -87.765400451)"
2,10224740,HY411595,09/05/2015 12:45:00 PM,035XX W BARRY AVE,2023,NARCOTICS,POSS: HEROIN(BRN/TAN),SIDEWALK,True,False,...,35.0,21.0,18,1152037.0,1920384.0,2015,02/10/2018 03:50:01 PM,41.937406,-87.716650,"(41.937405765, -87.716649687)"
3,10224741,HY411610,09/05/2015 01:00:00 PM,0000X N LARAMIE AVE,0560,ASSAULT,SIMPLE,APARTMENT,False,True,...,28.0,25.0,08A,1141706.0,1900086.0,2015,02/10/2018 03:50:01 PM,41.881903,-87.755121,"(41.881903443, -87.755121152)"
4,10224742,HY411435,09/05/2015 10:55:00 AM,082XX S LOOMIS BLVD,0610,BURGLARY,FORCIBLE ENTRY,RESIDENCE,False,False,...,21.0,71.0,05,1168430.0,1850165.0,2015,02/10/2018 03:50:01 PM,41.744379,-87.658431,"(41.744378879, -87.658430635)"


In [7]:
# Renaming columns
table_db = df.rename(columns={'Primary Type': 'Primary_Type', 'Location Description': 'Location_Description'}, errors='raise')

In [8]:
# Dropping columns
table_db = table_db[['Date', 'IUCR', 'Primary_Type', 'Description', 'Location_Description', 'Arrest', 'Domestic', 'Year', 'Latitude', 'Longitude']]    
table_db.head()

,Date,IUCR,Primary_Type,Description,Location_Description,Arrest,Domestic,Year,Latitude,Longitude
0,09/05/2015 01:30:00 PM,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,2015,41.815117,-87.670000
1,09/04/2015 11:30:00 AM,0870,THEFT,POCKET-PICKING,CTA BUS,False,False,2015,41.895080,-87.765400
2,09/05/2015 12:45:00 PM,2023,NARCOTICS,POSS: HEROIN(BRN/TAN),SIDEWALK,True,False,2015,41.937406,-87.716650
3,09/05/2015 01:00:00 PM,0560,ASSAULT,SIMPLE,APARTMENT,False,True,2015,41.881903,-87.755121
4,09/05/2015 10:55:00 AM,0610,BURGLARY,FORCIBLE ENTRY,RESIDENCE,False,False,2015,41.744379,-87.658431


In [9]:
# Adding months tables as Intiger and renaming columns
table_db['Month'] = table_db['Date'].str[0:2].astype(int)
table_db = table_db.rename(columns={'ID': 'id', 'Date': 'date', 'IUCR': 'iucr', 'Primary_Type': 'primary_type', 'Description': 'description', 'Location_Description': 'location_description', 'Arrest': 'arrest', 'Domestic': 'domestic', 'Year': 'year', "Latitude": "latitude", "Longitude": "longitude", "Month": "month"})
table_db.head()

,date,iucr,primary_type,description,location_description,arrest,domestic,year,latitude,longitude,month
0,09/05/2015 01:30:00 PM,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,2015,41.815117,-87.670000,9
1,09/04/2015 11:30:00 AM,0870,THEFT,POCKET-PICKING,CTA BUS,False,False,2015,41.895080,-87.765400,9
2,09/05/2015 12:45:00 PM,2023,NARCOTICS,POSS: HEROIN(BRN/TAN),SIDEWALK,True,False,2015,41.937406,-87.716650,9
3,09/05/2015 01:00:00 PM,0560,ASSAULT,SIMPLE,APARTMENT,False,True,2015,41.881903,-87.755121,9
4,09/05/2015 10:55:00 AM,0610,BURGLARY,FORCIBLE ENTRY,RESIDENCE,False,False,2015,41.744379,-87.658431,9


# Postgres connection - Chicago crime table

In [14]:
# Remember to change to your Postgres password
rds_connection_string = "postgres:postgres@localhost:5432/Project_03"

# Adding a connection if none are available
engine = create_engine(f'postgresql://{rds_connection_string}', pool_size=10, max_overflow=20)

In [15]:
table_db.to_sql(name='chicago_crime', con=engine, if_exists='append', index=False)

In [16]:
c = engine.execute('SELECT * FROM chicago_crime').fetchall()
print(c[:10])

[(1, '09/05/2015 01:30:00 PM', '0486', 'BATTERY', 'DOMESTIC BATTERY SIMPLE', 'RESIDENCE', False, True, 2015, 41.815117282, -87.669999562, 9), (2, '09/04/2015 11:30:00 AM', '0870', 'THEFT', 'POCKET-PICKING', 'CTA BUS', False, False, 2015, 41.895080471, -87.765400451, 9), (3, '09/05/2015 12:45:00 PM', '2023', 'NARCOTICS', 'POSS: HEROIN(BRN/TAN)', 'SIDEWALK', True, False, 2015, 41.937405765, -87.716649687, 9), (4, '09/05/2015 01:00:00 PM', '0560', 'ASSAULT', 'SIMPLE', 'APARTMENT', False, True, 2015, 41.881903443, -87.755121152, 9), (5, '09/05/2015 10:55:00 AM', '0610', 'BURGLARY', 'FORCIBLE ENTRY', 'RESIDENCE', False, False, 2015, 41.744378879, -87.658430635, 9), (6, '09/04/2015 06:00:00 PM', '0620', 'BURGLARY', 'UNLAWFUL ENTRY', 'RESIDENCE-GARAGE', False, False, 2015, 41.914635603, -87.681630909, 9), (7, '09/05/2015 01:00:00 PM', '0860', 'THEFT', 'RETAIL THEFT', 'GROCERY FOOD STORE', True, False, 2015, 41.851988885, -87.689219118, 9), (8, '09/05/2015 11:30:00 AM', '0320', 'ROBBERY', 'STR